In [1]:
import tensorflow as tf
from tensorflow.contrib import autograph #import autograph from contrib package to use graph mode
%load_ext tensorboard
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd 
import seaborn as sns
from scipy import stats 
from scipy import linalg

In [2]:
print(tf.__version__)

1.15.0


-----

**In-class Exercise**

1. Create a graph that produces a 3x4 matrix, where an op is added to the default graph and you:

    - add a node A as a matrix of shape [3,2] with some constants 
    - add a node B as a matrix opened to take variables of type float32 (use placeholder)
    - add an operation of their product to the graph producing an output matrix of shape [3,4]
    - print the resulting matrix on the screen

In [3]:
#node A
a = tf.constant(np.array([[1,2.],[3,4],[5,6.]], 
                         dtype=np.float32),
               name = "node_A")

#node B
b = tf.placeholder(dtype = tf.float32, name = "node_B")
product = tf.matmul(a,b)

with tf.Session() as sess:
    input_dict = {b: [[2.,6,9,2.],[8,2,2,4]]}
    result = sess.run(product, feed_dict = input_dict)
    print(result)



[[18. 10. 13. 10.]
 [38. 26. 35. 22.]
 [58. 42. 57. 34.]]


-----------------

**Eager Execution by Default in 2x** 


    a = tf.constant(np.array([1., 2., 3.]))
    b = tf.constant(np.array([4., 5., 6.]))

    c = tf.tensordot(a,b,1) #calculate the dot product

    #eager execution
    print(c.numpy())

**TensorFlow 1.x** 

In [4]:
a = tf.constant(np.array([1., 2., 3.]))
b = tf.constant(np.array([4., 5., 6.]))

c = tf.tensordot(a,b,1) #calculate the dot product

sess = tf.Session()
print(sess.run(c))


32.0


**Placeholders comparison**

In [5]:
f = tf.placeholder(dtype = tf.int32)

placeholder = int(input('Enter a number here: '))
#TF 1.X
sess = tf.Session()
outputs = sess.run(f, feed_dict={f: placeholder})
print(outputs)

sess.close()

Enter a number here: 4
4


**Recap: Making a graph in 1.X**

In [6]:

# Defining our input nodes
a = tf.constant(5, name = "input_a")
b = tf.constant(3, name = "input_b")

#Defining the next two nodes in our graph
c = tf.multiply(a,b, name ="mul_c")
d = tf.add(a,b, name="add_d")

#This last line defines the final node in our graph:
e = tf.add(c,d, name = "add_e")

#To run we have to add the two extra lines or run them in the shell
sess = tf.Session()
sess.run(e)

# To create the graph
sess.graph.as_graph_def()
file_writer = tf.summary.FileWriter('./graphs', sess.graph)

#We clean up before we exit:
file_writer.close()


In [7]:
!ls graphs

events.out.tfevents.1582681850.Nhans-MacBook-Pro.local
events.out.tfevents.1582839111.JIMMYSPC
events.out.tfevents.1582840175.JIMMYSPC
events.out.tfevents.1582844030.JIMMYSPC
events.out.tfevents.1582844379.JIMMYSPC
events.out.tfevents.1582844562.JIMMYSPC
events.out.tfevents.1582847543.JIMMYSPC
events.out.tfevents.1582847625.JIMMYSPC
events.out.tfevents.1582848119.JIMMYSPC


In [8]:
%tensorboard --logdir='./graphs/events.out.tfevents.1582844562.JIMMYSPC'/ --port=6006


Reusing TensorBoard on port 6006 (pid 8584), started 0:46:48 ago. (Use '!kill 8584' to kill it.)

-----

**Autograph**
- is used to convert a function into TensorFlow function

In [9]:
#function to find the next number in fibonacci series

def fib1(a):
    x,y = 0, 1
    for i in range(a-1):
        x,y = y, x + y
    return y

tf_fib1 = autograph.to_graph(fib1) #using autograph we convert the function to a tensorflow function

graph = tf.Graph() #create a graph 
with graph.as_default():
    #within the graph we call the 'fib1' function so it runs in graph mode 
    answer = tf_fib1(tf.constant(8)) #only created the operations required to perform the computation and adds them to the graph

with tf.Session(graph=graph) as sess: #start a session to evaluate the result in tensor and print
    print(answer.eval())
    


21


-----
**Initializing Variables**

- in tensorflow before execution all global variables are required to be initialized
- everything except of variables do not require intialization (constants and placeholders)
- but every used variables (even if it is a constant) must be initialized

        init = tf.global_variable_initializer()

-------
**Downloading datasets in tensorflow for machine learning**
  
        import tensorflow_datasets as tfds
        tf_flowers = tfds.load(name="tf_flowers", split ="train")

-----
**Namescopes**

In [10]:
with tf.name_scope("My_Graph"):
    with tf.name_scope("Scope_1"):
        a = tf.add(1,2, name ="addition_1")
        b = tf.multiply(a, 3, name ="multiplying_1")
    with tf.name_scope("Scope_2"):
        c = tf.add(4,5 ,name ="addition_2")
        d = tf.multiply(c, 6, name ="multiplying_1")
    with tf.name_scope("Scope_3"):
        e = tf.multiply(4,5)
        f = tf.div(c,6, name="division")

g = tf.add(b,d)
h = tf.multiply(g, f)

#We have defined a computional graph now, we need to execute it
with tf.Session() as sess:
    sess = tf.Session()
    print(sess.run(h))
    writer = tf.summary.FileWriter('./name_scopes/name_scope_1', graph = tf.get_default_graph())
    writer.close()

63


In [11]:
!ls name_scopes

name_scope_1
name_scope_2
name_scope_3


In [12]:
%tensorboard --logdir='./name_scopes/name_scope_1' --port=8008


Reusing TensorBoard on port 8008 (pid 8100), started 0:52:00 ago. (Use '!kill 8100' to kill it.)

In [13]:
with tf.name_scope('Scope_A'):
    a = tf.add(1,2, name="A_add")
    b = tf.multiply(a,3, name="A_mul")
with tf.name_scope("Scope_B"):
    c = tf.add(4,5, name ="B_add")
    d = tf.multiply(c,6, name ="B_mul")
    
e = tf.add(b,d, name="output")


sess = tf.Session()
print(sess.run(h))
writer = tf.summary.FileWriter('./name_scopes/name_scope_2', graph = tf.get_default_graph())
writer.close()

63


In [14]:
!ls ./name_scopes

name_scope_1
name_scope_2
name_scope_3


In [15]:
%tensorboard --logdir='./name_scopes/name_scope_2' --port=8009

Reusing TensorBoard on port 8009 (pid 10472), started 0:07:50 ago. (Use '!kill 10472' to kill it.)

**Nesting Namescopes within other name scopes**

In [19]:
graph = tf.Graph()

with graph.as_default():
    in_1 = tf.constant(3, dtype = tf.float32, name ="input_1") #placeholder(tf.float32, shape =[], name="input_a")
    in_2 = tf.constant(2, dtype = tf.float32, name ="input_2") #placeholder(tf.float32, shape =[], name="input_b")
    const = tf.constant(3, dtype = tf.float32, name ="static_value")
    
    with tf.name_scope("Transformation"):
        with tf.name_scope("A"):
            A_mul = tf.multiply(in_1, const)
            A_out = tf.subtract(A_mul, in_1)
        
        with tf.name_scope("B"):
            B_mul = tf.multiply(in_2, const)
            B_out = tf.subtract(B_mul, in_2)
        
        with tf.name_scope("C"):
            C_div = tf.div(A_out, B_out)
            C_out = tf.add(C_div, const)
    
        with tf.name_scope("D"):
            D_div = tf.div(B_out, A_out)
            D_out = tf.add(D_div, const)
    
    out = tf.maximum(C_out, D_out)

writer = tf.summary.FileWriter('./name_scopes/name_scope_3', graph =graph)
writer.close()

In [20]:
%tensorboard --logdir='./name_scopes/name_scope_3' --port=8012

Reusing TensorBoard on port 8012 (pid 10820), started 0:09:51 ago. (Use '!kill 10820' to kill it.)